In [1]:
from geopy.geocoders import Nominatim
import pandas as pd

geolocator = Nominatim(user_agent="mpplication")
location = geolocator.geocode("Краснодар Адмирала Крузенштерна 5к1")
print((location.latitude, location.longitude))

(45.0927545, 39.040469)


In [2]:
data_frame_all = pd.read_csv('krasnodar_flats.csv') 

In [3]:
df = data_frame_all[(data_frame_all['location'].notna()) & (data_frame_all['street'].notna()) & (data_frame_all['house_number'].notna())]

In [4]:
df = df[['location', 'street', 'house_number', 'total_meters', 'price']]

In [5]:
df

,location,street,house_number,total_meters,price
1,Краснодар,Школьная,1,39.00,7800000
2,Краснодар,Ярославская,115лит3,41.60,6539520
6,Краснодар,Уральская,100/8,39.05,5490430
8,Краснодар,Старокубанская,123А,43.40,9711618
9,Краснодар,Лиссабонская,80,30.10,3300000
...,...,...,...,...,...
4550,Краснодар,Григория Булгакова,10,77.00,8600000
4551,Краснодар,Ставропольская,18,74.00,14950000
4552,Краснодар,Черкасская,79,88.30,9720000
4553,Краснодар,Им. Героя Сарабеева В.И.,9/1,42.21,5400000


In [6]:
import numpy as np

In [7]:
# NumPy написан на языке программирования C, что позволяет ему выполнять операции гораздо быстрее, чем чистый Python. 
# Многие функции NumPy используют высокоэффективные алгоритмы, 
# реализованные на низком уровне, что снижает накладные расходы на выполнение операций.

In [9]:
array = df.to_numpy()  # для производительности пере

In [10]:
# Создание пустого DataFrame с двумя столбцами
df = pd.DataFrame(columns=['geo_lat', 'geo_lon', 'total_meters', 'price' ])

In [11]:
for row in array:
    try:
        location = geolocator.geocode(f"{row[0]} {row[1]} {row[2]}")
    except Exception as err:
        print("Ошибка", err)
    else:
        try:
            # Добавление значений в DataFrame
            df.loc[len(df)] = [location.latitude, location.longitude, row[3], row[4]]
        except Exception as err:
            print("Ошибка", err)

Ошибка 'NoneType' object has no attribute 'latitude'
Ошибка 'NoneType' object has no attribute 'latitude'
Ошибка HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B4%D0%B0%D1%80+%D0%A3%D1%80%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F++87%2F7&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Ошибка 'NoneType' object has no attribute 'latitude'
Ошибка HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B4%D0%B0%D1%80+%D0%A3%D1%80%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F++87%2F7&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


KeyboardInterrupt: 

In [12]:
df 

,geo_lat,geo_lon,total_meters,price
0,43.673259,40.262580,39.00,7800000.0
1,45.037277,39.088344,39.05,5490430.0
2,45.017370,39.045940,43.40,9711618.0
3,44.996288,39.068140,36.30,4500000.0
4,45.139436,38.987286,33.70,3400000.0
5,45.093145,39.041124,39.10,4887500.0
6,45.092772,38.904856,34.00,5500000.0
7,45.089686,39.023104,36.10,4151500.0
8,45.009231,38.996094,56.07,7401000.0


In [13]:
df['price_per_meter'] = df['price'] / df['total_meters']

In [14]:
df1 = df

In [15]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
# Функция для выбора цвета в зависимости от значения переменной
def get_color(value, cmap='viridis'):
    norm = mcolors.Normalize(vmin=df['price_per_meter'].min(), vmax=df['price_per_meter'].max())
    colormap = plt.get_cmap(cmap)
    return mcolors.to_hex(colormap(norm(value)))

# Добавление столбца с цветами в DataFrame
df['color'] = df['price_per_meter'].apply(lambda x: get_color(x))

In [38]:
df.to_csv('geolocation_data2.csv', index=False) 

In [16]:
# Функция для интерполяции цветов
def interpolate_color(value, vmin, vmax, cmap):
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    return mcolors.rgb2hex(cmap(norm(value)))

# Минимальное и максимальное значения цены
min_price = df['price'].min()
max_price = df['price'].max()

# Применение функции к DataFrame для добавления нового столбца с интерполированными цветами
df['color'] = df['price'].apply(lambda x: interpolate_color(x, min_price, max_price, plt.cm.RdYlGn))


In [18]:
df.head()

,geo_lat,geo_lon,total_meters,price,price_per_meter,color
0,43.673259,40.262580,39.00,7800000.0,200000.000000,#a7d96b
1,45.037277,39.088344,39.05,5490430.0,140600.000000,#fdbd6d
2,45.017370,39.045940,43.40,9711618.0,223770.000000,#006837
3,44.996288,39.068140,36.30,4500000.0,123966.942149,#ec5c3b
4,45.139436,38.987286,33.70,3400000.0,100890.207715,#a50026


In [40]:
df.to_csv('geolocation_data.csv', index=False)